# Тестирование работы API для тегирования видео

In [1]:
%pip install pandas httpx --upgrade --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

## Загружаем тестовую выборку

In [5]:
x_test = pd.read_csv("sample_submission.csv")
x_test

,video_id,title,description,predicted_tags
0,c00d1b173ed8680c155b773d43af7791,Тяпа | #24 | 8 МАРТА. ТЯПА ПОЗДРАВЛЯЕТ ЛИЗУ. З...,Утро Тяпы начинается с недвусмысленного напоми...,NaN
1,6064c4511a067f2e1d1cef81c21421e8,БЮРО ДОБРЫХ ДЕЛ №5 | ХУДОЖНИК ИЗ АСТРАХАНИ | К...,Добро пожаловать на канал “Бюро Добрых Дел”! Е...,NaN
2,3071e2054a02a03150463e514f05fe90,Такой расклад. Эфир 17. Таро. Про личные качес...,В этом выпуске шоу «Такой расклад» вы увидите ...,NaN
3,60867179613376c8b56f0705114a7685,TREXTRAVELER в ГрандТуре «Байкальская миля 202...,"Визит в Государственный природный заказник ""Су...",NaN
4,706f44ef4711eaee99035260c2d1ab86,Тру ДЕТЕКТОР I #5,Вы смотрите шоу с детектором лжи «Тру ДЕТЕКТОР...,NaN
...,...,...,...,...
194,0f84219f46782fca9482fb88da146ed3,Диктант со звездой | Выпуск 7 | Чипинкос,В этом выпуске - рэпер Чипинкос останется один...,NaN
195,2f67917a65dd8d5b348de81d15b1d8ff,Полупановы. «Эпизод 16»,В этом эпизоде Игорь готовится к любимому праз...,NaN
196,6fa39a883591bc0e2cf7ef6c5349c9df,"Салют, Начальник! 2 сезон, 6 серия",Назаров устраивает майору «сюрприз» прямо за с...,NaN
197,bff493ac045715566a86cd7e84051938,Абьюз-шоу |Выпуск №7 часть 1I Алина Егорова,Героиней этого выпуска «Абьюз шоу» стала Алина...,NaN


## Обращаемся к API

In [20]:
import httpx
import urllib

sample_submission = []
for i, data in x_test.iterrows():

    with open(f"videos/{data['video_id']}.mp4", "rb") as video:
        files = {'video': video}
        tags = httpx.post(f"http://localhost/predict_tokens?title={urllib.parse.quote_plus(data['title'])}&description={urllib.parse.quote_plus(data['description'])}", files=files, timeout=300)
        print(i, tags.text)
    sample_submission.append({
        "video_id": data["video_id"],
        "predicted_tags": tags.text,
    })

sample_submission = pd.DataFrame(sample_submission)
sample_submission.predicted_tags = sample_submission.predicted_tags.apply(lambda x: "[]" if "detail" in x else x)
sample_submission

0 ["Массовая культура: Юмор и сатира","Фильмы и анимация: Комедии","Телевидение: Комедийные каналы"]
1 ["Новости и политика: Международные новости"]
2 ["Религия и духовность: Астрология","Хобби и интересы: Паранормальные явления","Наука: Космос и астрономия"]
3 ["Путешествия: Тип путешествия: Путешествия на машине","Путешествия: Тип путешествия: Активный туризм","События и достопримечательности: Парки и природные объекты"]
4 ["Массовая культура: Юмор и сатира","Фильмы и анимация: Комедии","Телевидение: Комедийные каналы"]
5 ["Музыка и аудио: Рок музыка: Легкий рок","Музыка и аудио: Рок музыка: Классический рок","Музыка и аудио: Поп-музыка"]
6 ["Транспорт: Видео с видеорегистраторов","События и достопримечательности: Исторические места и достопримечательности","Спорт: Автогонки: Мотоспорт"]
7 ["Музыка и аудио: Разное (Музыка и аудио)","Музыка и аудио: Комедия и стендап (Музыка и аудио)","Музыка и аудио: Радио: Общественное радиовещание"]
8 ["Массовая культура: Юмор и сатира","Фильмы и а

,video_id,predicted_tags
0,c00d1b173ed8680c155b773d43af7791,"[""Массовая культура: Юмор и сатира"",""Фильмы и ..."
1,6064c4511a067f2e1d1cef81c21421e8,"[""Новости и политика: Международные новости""]"
2,3071e2054a02a03150463e514f05fe90,"[""Религия и духовность: Астрология"",""Хобби и и..."
3,60867179613376c8b56f0705114a7685,"[""Путешествия: Тип путешествия: Путешествия на..."
4,706f44ef4711eaee99035260c2d1ab86,"[""Массовая культура: Юмор и сатира"",""Фильмы и ..."
...,...,...
194,0f84219f46782fca9482fb88da146ed3,"[""Массовая культура: Юмор и сатира"",""Фильмы и ..."
195,2f67917a65dd8d5b348de81d15b1d8ff,"[""Массовая культура: Юмор и сатира"",""Хобби и и..."
196,6fa39a883591bc0e2cf7ef6c5349c9df,"[""Фильмы и анимация: Комедии"",""Массовая культу..."
197,bff493ac045715566a86cd7e84051938,"[""Массовая культура: Беременности знаменитосте..."


In [30]:
for_submit = pd.concat([x_test[["video_id", "title", "description"]], sample_submission], axis = 1)
for_submit.T.drop_duplicates().T.to_csv("first.csv")

## Загружаем целевые метки

In [29]:
ground_truth = pd.read_csv('y_test.csv', index_col=0)
ground_truth = ground_truth.join(x_test, how='inner')

## Запускаем метрику

In [28]:
import pandas as pd
import argparse
import ast
import numpy as np

# #sample_submission = pd.read_csv('/content/sample_submission.csv')
# sample_submission = pd.DataFrame([['asd', 'name', 'disc', 'tag1']], columns=['video_id', 'title', 'discription', 'tags'])
# sample_submission['predicted_tags'] = np.repeat(["['Карьера: Cтажировки', 'Карьера: Составление резюме']"], len(sample_submission))

def iou_metric(ground_truth, predictions):
    iou =  len(set.intersection(set(ground_truth), set(predictions)))
    iou = iou/(len(set(ground_truth).union(set(predictions))))
    return iou

def split_tags(tag_list):
    final_tag_list = []
    for tag in tag_list:
        print(tag)
        tags = tag.split(": ")
        if len(tags) == 3:
            final_tag_list.append(tags[0])
            final_tag_list.append(tags[0] + ": " + tags[1])
            final_tag_list.append(tags[0]+ ": " + tags[1] + ": " + tags[2])
        elif len(tags) == 2:
            final_tag_list.append(tags[0])
            final_tag_list.append(tags[0] + ": " + tags[1])
        elif len(tags) == 1:
            final_tag_list.append(tags[0])
        else:
            print("NOT IMPLEMENTED!!!!", tag)
    print()
    return final_tag_list


def find_iou_for_sample_submission(pred_submission, true_submission):
    ground_truth_df = true_submission
    ground_truth_df["tags"] = ground_truth_df["tags"].apply(lambda l: l.split(', '))
    ground_truth_df["tags_split"] = ground_truth_df["tags"].apply(lambda l: split_tags(l))

    predictions_df = pred_submission
    predictions_df["predicted_tags"] = predictions_df["predicted_tags"].apply(ast.literal_eval)
    predictions_df["predicted_tags_split"] = predictions_df["predicted_tags"].apply(lambda l: split_tags(l))
    iou=0
    counter = 0
    for i, row in ground_truth_df.iterrows():
        predicted_tags = predictions_df[predictions_df["video_id"]==row["video_id"]]["predicted_tags_split"].values[0]
        iou_temp=iou_metric(row['tags_split'], predicted_tags)
        iou+=iou_temp
        counter+=1

    return iou/counter

# ground_truth = pd.DataFrame([['asd']], columns=['video_id'])
# ground_truth['tags'] = np.repeat([["Карьера: Cтажировки, Карьера: Составление резюме"]], len(ground_truth))

if __name__ == '__main__':

    try:
        pred_submission = sample_submission #pd.read_csv(pred_path, sep = ',')
    except Exception:
        assert False, 'Ошибка при загрузке решения участника'
    try:
        true_submission = ground_truth #pd.read_csv(true_path, sep = ',')
    except Exception:
        assert False, 'Ошибка при загрузке эталонного решения'


    final_score = find_iou_for_sample_submission(pred_submission, true_submission)
    print("FINAL_SCORE:", final_score) #final_score)

Семья и отношения: Брак и гражданские союзы
Массовая культура

Спорт: Борьба
Массовая культура

Массовая культура: Юмор и сатира

Дом и сад: Дизайн интерьера

Транспорт
Спорт: Автогонки
События и достопримечательности: Спортивные события

Спорт: Рыбалка

Массовая культура: Юмор и сатира

Массовая культура: Юмор и сатира
Семья и отношения

Личные финансы
Карьера

Дом и сад: Дизайн интерьера

Стиль и красота: Женская мода

Массовая культура: Юмор и сатира
Транспорт

Массовая культура: Юмор и сатира

Массовая культура: Юмор и сатира

Спорт
Массовая культура: Юмор и сатира

Массовая культура: Отношения знаменитостей

Массовая культура: Юмор и сатира

Массовая культура: Юмор и сатира

Спорт: Рыбалка

Массовая культура: Юмор и сатира
Семья и отношения

Массовая культура
Спорт

Музыка и аудио: Комедия и стендап (Музыка и аудио)
События и достопримечательности: Концерты и музыкальные мероприятия

Массовая культура: Скандалы знаменитостей

Массовая культура: Юмор и сатира

Религия и духовность: